### Preprocessing of the Twitter Dataset

In [1]:
import pandas as pd
import sys
sys.path.append("..")
from tqdm import tqdm_notebook

In [2]:
import glob

files = glob.glob("*")

files

['twitter_dataset.csv',
 'data.p',
 'twitter_pre.ipynb',
 'twitter_pre-Copy1.ipynb',
 'tfidf.p',
 'adr_dataset_split.csv']

In [3]:
tweets = pd.read_csv('adr_dataset_split.csv', sep = ",")

tweets.columns

Index(['text', 'label', 'exp_split'], dtype='object')

In [4]:
from preprocess_bc import cleaner

In [ ]:
tweets["text"] = tweets["text"].apply(lambda x: " ".join(cleaner(x)))

In [ ]:
tweets.head(10).text[0]

In [ ]:
from preprocess_bc import extract_vocabulary_

word_to_ix = extract_vocabulary_(min_df = 2, dataframe = tweets)

In [ ]:
tweets["text"] = tweets["text"].apply(lambda x: ("<SOS> " + x + " <EOS>").split())

### Forming our vocabulary, ixing data and saving it

In [ ]:
from preprocess_bc import text_to_seq

ix_to_word = {v:k for k,v in word_to_ix.items()}

train_ix = text_to_seq(tweets[tweets.exp_split == "train"][["text","label"]].values, word_to_ix)
dev_ix = text_to_seq(tweets[tweets.exp_split == "dev"][["text","label"]].values, word_to_ix)
test_ix = text_to_seq(tweets[tweets.exp_split == "test"][["text","label"]].values, word_to_ix)

In [ ]:
train_ix[0]

### Preparing our embeddings

In [ ]:
from preprocess_bc import pretrained_embeds, DataHolder_BC

pre = pretrained_embeds("fasttext.simple.300d", ix_to_word)

pretrained = pre.processed()

In [ ]:
data = DataHolder_BC(train_ix, dev_ix, test_ix, word_to_ix, embeds =  pretrained)

In [ ]:
import pickle

pickle.dump(data, open("data.p", "wb"))